## Descargar librerias

In [1]:
# importar librerias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

In [2]:
### para graficar localmente
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

init_notebook_mode(connected=True)

## Cargar datos y modificar

Los datos están divididos en dos tablas, uno para producción primaria y otro para producción secundaria (productos procesados), según aparece en los documentos de balances energéticos. En el portal de datos abiertos hay datos para más años de los que incluyo. En principio busqué los datos de 1999-2017 (último año disponible). Extendí la base hasta 1990 porque estaba en el archivo que estaba usando. Ahora, dado que es un trabajo manual y mi foco está en los años recientes, no incluí toda la serie histórica porque en lo inmediato no me servía pero me hubiera implicado más tiempo de carga de datos manual del que tengo. A futuro se puede agregar.

In [3]:
# cargar datos
df_primaria = pd.read_csv('insumos/hidrocarburos_produccion-importacion_primaria_1990-2017.csv', sep=";")

# ordeno por año para que el gráfico no se distorsione
df_primaria.sort_values(by='anio', ascending=False, inplace=True)

# opcional, para limitar la serie a los años inlcuidos en el primer posteo
# df_primaria.drop(df_primaria[df_primaria.anio < 1999].index, inplace=True)

In [4]:
df_primaria.columns

Index(['anio', 'producto', 'produccion_local', 'importado', 'exportado',
       'oferta_bruta', 'importado_pcnt', 'exportado_pcnt'],
      dtype='object')

In [5]:
df_primaria.producto.value_counts()

Gas Natural de Pozo    28
Petroleo               28
Name: producto, dtype: int64

In [6]:
# cargar datos
df_secundaria = pd.read_csv('insumos/hidrocarburos_produccion-importacion_secundaria_1990-2017.csv', sep=";")
df_secundaria['exportado'] = df_secundaria['exportado']*(-1) 

# los datos están desagregados por sub_producto, lo agrego según si son derivados de petróleo o de gas
df_sec = (
    df_secundaria.groupby(['anio', 'categoria_derivados'], as_index=False)
    [['produccion_local', 'importado','exportado']]
    .sum()
)

# vuelvo a calcular las columnas con los valores agregados
df_sec['oferta_bruta'] = df_sec.produccion_local + df_sec.importado
df_sec['importado_pcnt'] = df_sec.importado / df_sec.oferta_bruta *100
df_sec['exportado_pcnt'] = df_sec.exportado / df_sec.oferta_bruta *100

In [7]:
df_sec.columns

Index(['anio', 'categoria_derivados', 'produccion_local', 'importado',
       'exportado', 'oferta_bruta', 'importado_pcnt', 'exportado_pcnt'],
      dtype='object')

## Producción primaria

In [25]:
produccion_local_pet = go.Scatter(
    x=df_primaria.query('producto == "Petroleo"').anio,
    y=df_primaria.query('producto == "Petroleo"').produccion_local,
    name='Producción Local'
)

oferta_bruta_pet = go.Scatter(
    x=df_primaria.query('producto == "Petroleo"').anio,
    y=df_primaria.query('producto == "Petroleo"').oferta_bruta,
    name='Oferta Bruta'
)

importado_pet = go.Bar(
    x=df_primaria.query('producto == "Petroleo"').anio,
    y=df_primaria.query('producto == "Petroleo"').importado_pcnt,
    yaxis='y2',
    name='% Importado'
)

exportado_pet = go.Bar(
    x=df_primaria.query('producto == "Petroleo"').anio,
    y=df_primaria.query('producto == "Petroleo"').exportado_pcnt,
    yaxis='y2',
    name='% Exportado'
)

data = [produccion_local_pet,oferta_bruta_pet, exportado_pet,importado_pet]

layout = go.Layout(
    title='Balance Energético: Petróleo (Sector Primario)',
    yaxis=dict(
        title='Oferta (TEP)',
        range=[0, 45000],
    ),
    yaxis2=dict(
        title='Importación y Exportación (%)',
        overlaying='y',
        side='right',
        range=[0, 60]
    ),
    xaxis=dict(
        tickmode='linear',
        ticks='inside',
        tick0=1990,
        dtick=5
    ),
    legend=dict(orientation="h")
)
fig = go.Figure(data=data, layout=layout)

# configurar para que se vea bien en celulares
fig['layout'].update(height=300, width=400)
fig['layout']['margin'].update(l=25, r=25, b=50, t=50, pad=0)
fig['layout']['yaxis'].update(automargin=True, titlefont=dict(size=10))
fig['layout']['yaxis2'].update(automargin=True, titlefont=dict(size=10))
fig['layout']['yaxis'].update(range=[0,50000])
fig['layout']['title'].update(font=dict(size=12))

# visualizar localmente
#iplot(fig, filename='balance-energetico-petroleo-primario')

# publicar
py.iplot(fig, filename='balance-energetico-petroleo-primario', sharing='public')

/usr/local/lib/python3.6/dist-packages/IPython/core/display.py:689: UserWarning:

Consider using IPython.display.IFrame instead



In [26]:
produccion_local_gas = go.Scatter(
    x=df_primaria.query('producto == "Gas Natural de Pozo"').anio,
    y=df_primaria.query('producto == "Gas Natural de Pozo"').produccion_local,
    name='Producción Local'
)

oferta_bruta_gas = go.Scatter(
    x=df_primaria.query('producto == "Gas Natural de Pozo"').anio,
    y=df_primaria.query('producto == "Gas Natural de Pozo"').oferta_bruta,
    name='Oferta Bruta'
)

importado_gas = go.Bar(
    x=df_primaria.query('producto == "Gas Natural de Pozo"').anio,
    y=df_primaria.query('producto == "Gas Natural de Pozo"').importado_pcnt,
    yaxis='y2',
    name='% Importado'
)

exportado_gas = go.Bar(
    x=df_primaria.query('producto == "Gas Natural de Pozo"').anio,
    y=df_primaria.query('producto == "Gas Natural de Pozo"').exportado_pcnt,
    yaxis='y2',
    name='% Exportado'
)

data = [produccion_local_gas,oferta_bruta_gas, exportado_gas,importado_gas]

layout = go.Layout(
    title='Balance Energético: Gas (Sector Primario)',
    yaxis=dict(
        title='Oferta (TEP)',
        range=[0, 50000],
    ),
    yaxis2=dict(
        title='Importación y Exportación (%)',
        overlaying='y',
        side='right',
        range=[0, 60]
    ),
    xaxis=dict(
        tickmode='linear',
        ticks='inside',
        tick0=1990,
        dtick=5
    ),
    legend=dict(orientation="h")
)
fig = go.Figure(data=data, layout=layout)

# configurar para que se vea bien en celulares
fig['layout'].update(height=300, width=400)
fig['layout']['margin'].update(l=25, r=25, b=50, t=50, pad=0)
fig['layout']['yaxis'].update(automargin=True, titlefont=dict(size=10))
fig['layout']['yaxis2'].update(automargin=True, titlefont=dict(size=10))
fig['layout']['yaxis'].update(range=[0,50000])
fig['layout']['title'].update(font=dict(size=12))

# visualizar localmente
#iplot(fig, filename='balance-energetico-gas-primario')

# publicar
py.iplot(fig, filename='balance-energetico-gas-primario', sharing='public')

## Producción Secundaria

In [27]:
produccion_local_pet = go.Scatter(
    x=df_sec.query('categoria_derivados == "petroleo"').anio,
    y=df_sec.query('categoria_derivados == "petroleo"').produccion_local,
    name='Producción Local'
)

oferta_bruta_pet = go.Scatter(
    x=df_sec.query('categoria_derivados == "petroleo"').anio,
    y=df_sec.query('categoria_derivados == "petroleo"').oferta_bruta,
    name='Oferta Bruta'
)

importado_pet = go.Bar(
    x=df_sec.query('categoria_derivados == "petroleo"').anio,
    y=df_sec.query('categoria_derivados == "petroleo"').importado_pcnt,
    yaxis='y2',
    name='% Importado'
)

exportado_pet = go.Bar(
    x=df_sec.query('categoria_derivados == "petroleo"').anio,
    y=df_sec.query('categoria_derivados == "petroleo"').exportado_pcnt,
    yaxis='y2',
    name='% Exportado'
)

data = [produccion_local_pet,oferta_bruta_pet, exportado_pet,importado_pet]

layout = go.Layout(
    title='Balance Energético: Petróleo (Sector Secundario)',
    yaxis=dict(
        title='Oferta (TEP)',
        range=[0, 45000],
    ),
    yaxis2=dict(
        title='Importación y Exportación (%)',
        overlaying='y',
        side='right',
        range=[0, 60]
    ),
    xaxis=dict(
        tickmode='linear',
        ticks='inside',
        tick0=1990,
        dtick=5
    ),
    legend=dict(orientation="h")
)
fig = go.Figure(data=data, layout=layout)

# configurar para que se vea bien en celulares
fig['layout'].update(height=300, width=400)
fig['layout']['margin'].update(l=25, r=25, b=50, t=50, pad=0)
fig['layout']['yaxis'].update(automargin=True, titlefont=dict(size=10))
fig['layout']['yaxis2'].update(automargin=True, titlefont=dict(size=10))
fig['layout']['yaxis'].update(range=[0,50000])
fig['layout']['title'].update(font=dict(size=12))

# visualizar localmente
#iplot(fig, filename='balance-energetico-petroleo-secundario')

# publicar
py.iplot(fig, filename='balance-energetico-petroleo-secundario', sharing='public')

In [28]:
produccion_local_gas = go.Scatter(
    x=df_sec.query('categoria_derivados == "gas"').anio,
    y=df_sec.query('categoria_derivados == "gas"').produccion_local,
    name='Producción Local'
)

oferta_bruta_gas = go.Scatter(
    x=df_sec.query('categoria_derivados == "gas"').anio,
    y=df_sec.query('categoria_derivados == "gas"').oferta_bruta,
    name='Oferta Bruta'
)

importado_gas = go.Bar(
    x=df_sec.query('categoria_derivados == "gas"').anio,
    y=df_sec.query('categoria_derivados == "gas"').importado_pcnt,
    yaxis='y2',
    name='% Importado'
)

exportado_gas = go.Bar(
    x=df_sec.query('categoria_derivados == "gas"').anio,
    y=df_sec.query('categoria_derivados == "gas"').exportado_pcnt,
    yaxis='y2',
    name='% Exportado'
)

data = [produccion_local_gas,oferta_bruta_gas, exportado_gas,importado_gas]

layout = go.Layout(
    title='Balance Energético: Gas (Sector Secundario)',
    yaxis=dict(
        title='Oferta (TEP)',
        range=[0, 45000],
    ),
    yaxis2=dict(
        title='Importación y Exportación (%)',
        overlaying='y',
        side='right',
        range=[0, 60]
    ),
    xaxis=dict(
        tickmode='linear',
        ticks='inside',
        tick0=1990,
        dtick=5
    ),
    legend=dict(orientation="h")
)
fig = go.Figure(data=data, layout=layout)

# configurar para que se vea bien en celulares
fig['layout'].update(height=300, width=400)
fig['layout']['margin'].update(l=25, r=25, b=50, t=50, pad=0)
fig['layout']['yaxis'].update(automargin=True, titlefont=dict(size=10))
fig['layout']['yaxis2'].update(automargin=True, titlefont=dict(size=10))
fig['layout']['yaxis'].update(range=[0,50000])
fig['layout']['title'].update(font=dict(size=12))

# visualizar localmente
#iplot(fig, filename='balance-energetico-gas-secundario')

# publicar
py.iplot(fig, filename='balance-energetico-gas-secundario', sharing='public')

## Detalle según derivados

In [29]:
df_secundaria.columns

Index(['anio', 'categoria_derivados', 'producto', 'produccion_local',
       'importado', 'exportado', 'oferta_bruta', 'importado_pcnt',
       'exportado_pcnt'],
      dtype='object')

In [30]:
df_secundaria.head(10)

,anio,categoria_derivados,producto,produccion_local,importado,exportado,oferta_bruta,importado_pcnt,exportado_pcnt
0,2016,gas,Gas Distribuido por Redes,34421.507090,4356.694070,50.417520,38778.201160,11.234905,0.146471
1,2016,gas,Gas de Refinería,1234.069150,0.000000,-0.000000,1234.069150,0.000000,0.000000
2,2016,gas,Gas Licuado,2864.554000,0.000000,967.375200,2864.554000,0.000000,33.770535
3,2016,petroleo,Gasolina Natural,946.153804,0.000000,213.304000,946.153804,0.000000,22.544326
4,2016,petroleo,Otras Naftas,810.496988,76.840608,502.227788,887.337596,8.659681,61.965411
5,2016,petroleo,Motonafta Total,6333.778858,185.185824,-0.000000,6518.964682,2.840724,0.000000
6,2016,petroleo,Kerosene y Aerokerosene,1377.604200,127.226400,973.197203,1504.830600,8.454533,70.644181
7,2016,petroleo,Diesel Oil + Gas Oil,9848.626500,3603.308561,74.990218,13451.935061,26.786544,0.761428
8,2016,petroleo,Fuel Oil,3844.429260,0.000000,640.485860,3844.429260,0.000000,16.660103
9,2015,gas,Gas Distribuido por Redes,33615.595539,4603.544370,72.967790,38219.139909,12.045128,0.217065


In [15]:
df_secundaria.producto.value_counts()

Fuel Oil                     28
Diesel Oil + Gas Oil         28
Otras Naftas                 28
Gasolina Natural             28
Motonafta Total              28
Kerosene y Aerokerosene      28
Gas Licuado                  28
Gas de Refinería             28
Gas Distribuido por Redes    28
Name: producto, dtype: int64

In [32]:
# Fuente de este codigo: https://stackoverflow.com/questions/50518090/python-plotly-multiple-dropdown-plots-each-of-which-have-subplots

# Creo figura
labels = ["Otras Naftas", 
          "Fuel Oil",
          "Kerosene y Aerokerosene",
          "Motonafta Total",
          "Diesel Oil + Gas Oil",
          "Gasolina Natural",
          "Gas de Refinería",
          "Gas Distribuido por Redes",
          "Gas Licuado"]

fig = tools.make_subplots(rows=1, 
                          cols=2, 
                          subplot_titles=('Importaciones', 'Exportaciones'), 
                          print_grid=False)

# Creo graficos de importacion
otras_naftas_imp = go.Bar(
    x=df_secundaria.query('producto == "Otras Naftas"').anio,
    y=df_secundaria.query('producto == "Otras Naftas"').importado,
    name='Otras Naftas',
    marker = dict(
        color='#0e0311'
    )
)

fuel_oil_imp = go.Bar(
    x=df_secundaria.query('producto == "Fuel Oil"').anio,
    y=df_secundaria.query('producto == "Fuel Oil"').importado,
    name='Fuel Oil',
    marker = dict(
        color='#370d45'
    )
)

kerosene_imp = go.Bar(
    x=df_secundaria.query('producto == "Kerosene y Aerokerosene "').anio,
    y=df_secundaria.query('producto == "Kerosene y Aerokerosene "').importado,
    name='Kerosene y Aerokerosene',
    marker = dict(
        color='#601679'
    )
)

motonafta_imp = go.Bar(
    x=df_secundaria.query('producto == "Motonafta Total "').anio,
    y=df_secundaria.query('producto == "Motonafta Total "').importado,
    name='Motonafta Total',
    marker = dict(
        color='#8920ac'
    )
)

diesel_imp = go.Bar(
    x=df_secundaria.query('producto == "Diesel Oil + Gas Oil"').anio,
    y=df_secundaria.query('producto == "Diesel Oil + Gas Oil"').importado,
    name='Diesel Oil + Gas Oil',
    marker = dict(
        color='#aa27d6'
    )
)

gasolina_imp = go.Bar(
    x=df_secundaria.query('producto == "Gasolina Natural"').anio,
    y=df_secundaria.query('producto == "Gasolina Natural"').importado,
    name='Gasolina Natural',
    marker = dict(
        color='#c364e3'
    )
)

gas_refineria_imp = go.Bar(
    x=df_secundaria.query('producto == "Gas de Refinería "').anio,
    y=df_secundaria.query('producto == "Gas de Refinería "').importado,
    name='Gas de Refinería',
    marker = dict(
        color='#441daf'
    )
)

gas_redes_imp = go.Bar(
    x=df_secundaria.query('producto == "Gas Distribuido por Redes"').anio,
    y=df_secundaria.query('producto == "Gas Distribuido por Redes"').importado,
    name='Gas Distribuido por Redes',
    marker = dict(
        color='#5423db'
    )
)

gas_licuado_imp = go.Bar(
    x=df_secundaria.query('producto == "Gas Licuado "').anio,
    y=df_secundaria.query('producto == "Gas Licuado "').importado,
    name='Gas Licuado',
    marker = dict(
        color='#8461e5'
    )
)

# Superpongo graficos para que funcione el efecto de la botonera
fig.append_trace(otras_naftas_imp, 1, 1)
fig.append_trace(fuel_oil_imp, 1, 1)
fig.append_trace(kerosene_imp, 1, 1)
fig.append_trace(motonafta_imp, 1, 1)
fig.append_trace(diesel_imp, 1, 1)
fig.append_trace(gasolina_imp, 1, 1)
fig.append_trace(gas_refineria_imp, 1, 1)
fig.append_trace(gas_redes_imp, 1, 1)
fig.append_trace(gas_licuado_imp, 1, 1)


# Creo graficos de exportacion
otras_naftas_exp = go.Bar(
    x=df_secundaria.query('producto == "Otras Naftas"').anio,
    y=df_secundaria.query('producto == "Otras Naftas"').exportado,
    name='Otras Naftas',
    marker = dict(
        color='#0e0311'
    )
)

fuel_oil_exp = go.Bar(
    x=df_secundaria.query('producto == "Fuel Oil"').anio,
    y=df_secundaria.query('producto == "Fuel Oil"').exportado,
    name='Fuel Oil',
    marker = dict(
        color='#370d45'
    )
)

kerosene_exp = go.Bar(
    x=df_secundaria.query('producto == "Kerosene y Aerokerosene "').anio,
    y=df_secundaria.query('producto == "Kerosene y Aerokerosene "').exportado,
    name='Kerosene y Aerokerosene',
    marker = dict(
        color='#601679'
    )
)

motonafta_exp = go.Bar(
    x=df_secundaria.query('producto == "Motonafta Total "').anio,
    y=df_secundaria.query('producto == "Motonafta Total "').exportado,
    name='Motonafta Total',
    marker = dict(
        color='#8920ac'
    )
)

diesel_exp = go.Bar(
    x=df_secundaria.query('producto == "Diesel Oil + Gas Oil"').anio,
    y=df_secundaria.query('producto == "Diesel Oil + Gas Oil"').exportado,
    name='Diesel Oil + Gas Oil',
    marker = dict(
        color='#aa27d6'
    )
)

gasolina_exp = go.Bar(
    x=df_secundaria.query('producto == "Gasolina Natural"').anio,
    y=df_secundaria.query('producto == "Gasolina Natural"').exportado,
    name='Gasolina Natural',
    marker = dict(
        color='#c364e3'
    )
)

gas_refineria_exp = go.Bar(
    x=df_secundaria.query('producto == "Gas de Refinería "').anio,
    y=df_secundaria.query('producto == "Gas de Refinería "').exportado,
    name='Gas de Refinería',
    marker = dict(
        color='#441daf'
    )
)

gas_redes_exp = go.Bar(
    x=df_secundaria.query('producto == "Gas Distribuido por Redes"').anio,
    y=df_secundaria.query('producto == "Gas Distribuido por Redes"').exportado,
    name='Gas Distribuido por Redes',
    marker = dict(
        color='#5423db'
    )
)

gas_licuado_exp = go.Bar(
    x=df_secundaria.query('producto == "Gas Licuado "').anio,
    y=df_secundaria.query('producto == "Gas Licuado "').exportado,
    name='Gas Licuado',
    marker = dict(
        color='#8461e5'
    )
)

fig.append_trace(otras_naftas_exp, 1, 2)
fig.append_trace(fuel_oil_exp, 1, 2)
fig.append_trace(kerosene_exp, 1, 2)
fig.append_trace(motonafta_exp, 1, 2)
fig.append_trace(diesel_exp, 1, 2)
fig.append_trace(gasolina_exp, 1, 2)
fig.append_trace(gas_refineria_exp, 1, 2)
fig.append_trace(gas_redes_exp, 1, 2)
fig.append_trace(gas_licuado_exp, 1, 2)

### Creo botonera 
buttons = []
   
buttons.append({'label': 'Petróleo',
                'method': 'update',
                'args': [{'visible': [True, True, True, True, True, True, False, False, False]},
                         {'title': 'Derivados del Petróleo'}]})
buttons.append({'label': 'Gas',
                'method': 'update',
                'args': [{'visible': [False, False, False, False, False, False, True, True, True]},
                         {'title': 'Derivados de Gas'}]})
buttons.append({'label': 'Todos',
                'method': 'update',
                'args': [{'visible': [True, True, True, True, True, True, True, True, True]},
                         {'title': 'Detalle de Derivados'}]})
    
updatemenus = list([
    dict(active=2,
        buttons=buttons,
         x=0.0,
         y=-0.5,
         xanchor = 'center',
         yanchor = 'bottom',
         direction="up",
         pad={'r':5, 't':5}
    )   
])

fig['layout']['title'] = 'Detalle de Derivados'
fig['layout']['showlegend'] = False
fig['layout']['updatemenus'] = updatemenus
fig['layout']['barmode'] = 'stack'

# configurar para que se vea bien en celulares
fig['layout'].update(height=300, width=400)
fig['layout']['margin'].update(l=25, r=25, b=50, t=50, pad=0)
#fig['layout']['xaxis'].update(domain=[0,0.47])
#fig['layout']['xaxis2'].update(domain=[0.53,1])
fig['layout']['yaxis'].update(title="TEP", automargin=True, titlefont=dict(size=10))
#fig['layout']['yaxis2'].update(title='miles de m3',automargin=True, titlefont=dict(size=10))
fig['layout']['title'].update(font=dict(size=12))
fig['layout']['yaxis'].update(range=[0,12000])
fig['layout']['yaxis2'].update(range=[0,12000])

# subplot title
for i in fig['layout']['annotations']:
    i['font'] = dict(size=10)


# Graficar localmente
#iplot(fig, filename='hidrocarburos-derivados')

# publicar
py.iplot(fig, filename='hidrocarburos-derivados', sharing='public')